In [1]:
#from run_test import *
from run import *
from functions import *

functions, game_info, game_state = load_variables("Shalan")

Using cache found in C:\Users\Kasper/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-10-16 torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3071.6875MB)

Fusing layers... 
Model Summary: 308 layers, 21037638 parameters, 0 gradients
Adding AutoShape... 


In [2]:
#game_info.save_automap_images = True
functions.start_diablo2()
functions.enter_game()
#functions.main_loop()

In [1]:
import itertools

def get_file_line_count(file_path):
    with open(file_path, "r") as file:
        return len(file.readlines())

def get_path_to_files_with_extensions(folder_path: str, file_extensions: list):
    return itertools.chain.from_iterable(
            map(lambda walk_tuple:
                map(lambda filename: os.path.join(walk_tuple[0], filename), 
                filter(lambda filename: os.path.splitext(filename)[1] in file_extensions, walk_tuple[2])),
            os.walk(folder_path))
            )

def get_sum_of_file_extensions_in_folder(file_function: callable, folder_path: str, file_extensions: list):
    return sum(map(file_function, get_path_to_files_with_extensions(folder_path, file_extensions)))


print(get_sum_of_file_extensions_in_folder(get_file_line_count, r"E:\Dropbox\Code\Diablo2bot", [".py"]))

4471


In [8]:
import win32api
import time
import win32gui
import win32con
from dataclasses_diablo import Position

route_name = "act_1_start_to_waypoint_3"

#click_positions = []
positions: list[Position] = []

state_left = win32api.GetKeyState(0x01)  # Left button down = 0 or 1. Button up = -127 or -128
state_right = win32api.GetKeyState(0x02)  # Right button down = 0 or 1. Button up = -127 or -128

state_ctrl = win32api.GetAsyncKeyState(win32con.VK_CONTROL)
last_switch_time = time.time()

while True:
    #a = win32api.GetKeyState(0x01)
    b = win32api.GetKeyState(0x02)

    a = win32api.GetAsyncKeyState(win32con.VK_CONTROL)

    if a != state_ctrl:  # Button state changed
        state_ctrl = a
        #if a < 0: # Left Button Pressed
        if time.time() - last_switch_time > 1:
            last_switch_time = time.time()
            if win32api.GetAsyncKeyState(win32con.VK_CONTROL): # Ctrl pressed
                mouse_position = win32gui.GetCursorPos()
                col = mouse_position[0] - functions.game_info.game_window_screen_location["left"]
                row = mouse_position[1] - functions.game_info.game_window_screen_location["top"]

                win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
                win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)

                print(col, row)
                positions.append(Position(row, col))
            else: # Left Button Released
                pass

    if b != state_right:  # Button state changed
        state_right = b
        if b < 0:
            print('Right Button Pressed')
            pass
            break
        else:
            print('Right Button Released')
    time.sleep(0.001)

file_path = os.path.join(functions.game_info.main_folder, "initialization_data", "fixed_routes", route_name + ".txt")
with open(file_path, "w") as file:
    file_content = ""
    for position in positions:
        file_content += f"{str(position.col)},{str(position.row)}\n"
    file.write(file_content)

777 153
728 383
Right Button Pressed


In [152]:
def read_csv(file_path: str, separator: str) -> tuple[np.ndarray, dict]:
    with open(file_path, "r") as file:
        lines = file.readlines()

        headers = lines[0].split(separator)
        headers[-1] = headers[-1][:-1] # Remove newline character from last column
        headers_indices_map = {header: i for i, header in enumerate(headers)}

        num_rows = len(lines)
        num_cols = len(headers)
        table = np.empty((num_rows, num_cols), dtype=object)

        for i, line in enumerate(lines[1:]):
            line_split = line.split(separator)
            line_split[-1] = line_split[-1][:-1] # Remove newline character from last column
            table[i] = line_split

    return table[0:-1, :], headers_indices_map

def pad_text(text: str, length: int) -> str:
    if text == "-1":
        return "-1"
    return "".join(["0" for _ in range(length - len(text))]) + text

def get_level_sprites(table, headers_indices_map) -> dict:
    level_sprites = {}
    level_names = []
    for i in range(table.shape[0]):
        level_name = table[i, headers_indices_map["LevelName"]]
        if level_name not in level_sprites:
            level_sprites[level_name] = set()
            level_names.append(level_name)

        sprite_ids = [
            table[i, headers_indices_map["Cel1"]],
            table[i, headers_indices_map["Cel2"]],
            table[i, headers_indices_map["Cel3"]],
            table[i, headers_indices_map["Cel4"]]
        ]

        for sprite_id in sprite_ids:
            if sprite_id != "-1":
                level_sprites[level_name].add(int(sprite_id))

    return level_sprites, level_names

def create_level_sprites_table(level_sprites: dict, level_names: list[str], num_sprites: int, headers: list[str]):
    num_rows = len(level_names) + 1
    num_cols = len(headers)
    table = np.empty((num_rows, num_cols), dtype=object)
    table[table == None] = "0"
    table[0, :] = headers

    for i, level_name in enumerate(level_names):
        #print(level_name)
        table[i+1, 0] = level_name
        for sprite_id in level_sprites[level_name]:
            table[i+1, sprite_id + 1] = "1"

    return table

def write_table_to_file(lines) -> None:
    with open("level_sprites.txt", "w") as file:
        file.writelines([line + "\n" for line in lines])

def table_to_lines(table: np.ndarray, separator: str):
    lines = []
    for i in range(table.shape[0]):
        lines.append(separator.join(table[i]))
    return lines

def get_table_headers(num_sprites: int) -> list[str]:
    headers = []
    headers.append("level_name")

    for sprite_id in range(num_sprites):
        headers.append(str(sprite_id))

    for sprite_id in range(num_sprites):
        headers.append(str(sprite_id))
    return headers

table, headers_indices_map = read_csv("initialization_data/AutoMap.txt", "\t")
level_sprites, level_names = get_level_sprites(table, headers_indices_map)
# level_sprites["1 Town"]
#level_names
num_sprites = len(os.listdir(os.path.join("initialization_data", "map_sprites")))
table_headers = get_table_headers(num_sprites)
level_sprites_table = create_level_sprites_table(level_sprites, level_names, num_sprites, table_headers)
write_table_to_file(table_to_lines(level_sprites_table, ","))

In [149]:
def read_level_sprites_table(file_path: str, separator: str, num_sprites: int):
    table_sprites = {}
    with open(file_path, "r") as file:
        lines = file.readlines()

        for line in lines:
            columns = line.split(separator)
            name = columns[0]
            has_sprite_ids = [i for i, has_sprite_id in enumerate(columns[1: num_sprites + 1]) if has_sprite_id == "1"]
            ignore_sprite_ids = [i for i, ignore_sprite_id in enumerate(columns[num_sprites + 1: ]) if ignore_sprite_id == "1"]
            table_sprites[name] = (has_sprite_ids, ignore_sprite_ids)

    return table_sprites
        
level_sprites = read_level_sprites_table("level_sprites.txt", ",", num_sprites)

In [243]:
s = time.time()
game_info.load_map_sprites()
print(time.time() - s)

0.49851179122924805


In [4]:
img = functions.take_screenshot()
zone_name = functions.get_current_zone(img)
img = remove_areas_from_image(img, game_info.static_areas)
automap, sprite_counts, window_indices = get_isolated_map(img, zone_name, game_info.area_mapping, game_info.zone_mapping, game_info.c_functions, game_info.transformation_array, game_info.window_offsets, game_info.transformation_array_back)
arr = np.zeros((600, 800*2, 3))
arr[0:600, 0:800, :] = img
arr[0:600, 800:800*2, :] = automap
cv2.imwrite("temp.png", arr)
os.startfile("temp.png")

In [5]:
img = functions.take_screenshot()
zone_name = functions.get_current_zone(img)
img = remove_areas_from_image(img, game_info.static_areas)
automap_, sprite_counts_, window_indices_ = get_isolated_map(img, zone_name, game_info.area_mapping, game_info.zone_mapping, game_info.c_functions, game_info.transformation_array, game_info.window_offsets, game_info.transformation_array_back)
arr = np.zeros((600, 800*2, 3))
arr[0:600, 0:800, :] = img
arr[0:600, 800:800*2, :] = automap_
cv2.imwrite("temp2.png", arr)
os.startfile("temp2.png")

In [60]:
rows = np.zeros(13916)
cols = np.zeros(13916)
c = 0
for row in range(0, 600 - 32, 4):
    for col in range(0, 800 - 16, 8):
        rows[c] = row   
        cols[c] = col   
        c += 1


def stitch_images_(automap_1, automap_2, sprites_counts_1, sprites_counts_2, window_indices_1, window_indices_2, rows, cols):
    arr = {}

    indices_1_offset = 0
    indices_2_offset = 0
    for sprite_counts_1, sprite_counts_2 in zip(sprites_counts_1, sprites_counts_2):
        #print(sprite_counts_1, sprite_counts_2)
        for i in range(indices_1_offset, indices_1_offset + sprite_counts_1):
            for j in range(indices_2_offset, indices_2_offset + sprite_counts_2):

                row_diff = rows[window_indices_1[i]] - rows[window_indices_2[j]]
                col_diff = cols[window_indices_1[i]] - cols[window_indices_2[j]]

                index_diff = (row_diff, col_diff)
                if index_diff not in arr:
                    arr[index_diff] = 1
                else:
                    arr[index_diff] += 1

        indices_1_offset += sprite_counts_1
        indices_2_offset += sprite_counts_2

    max_count = 0
    max_index_diff = 0
    for index_diff, count in arr.items():
        if count > max_count:
            max_count = count
            max_index_diff = index_diff

    print(max_count)
    print(max_index_diff)

    a = np.roll(automap, (-int(max_index_diff[0]), -int(max_index_diff[1]), 0), axis=(0,1,2))
    cv2.imwrite("2.png", a)
    os.startfile("2.png")
        


stitch_images_(automap, automap_, sprite_counts, sprite_counts_, window_indices, window_indices_, rows, cols)

353
(40.0, -8.0)


In [56]:
cv2.imwrite("bla.png", automap)
os.startfile("bla.png")

In [57]:
a = np.roll(automap, (0,10,0), axis=(0,1,2))
cv2.imwrite("bla2.png", a)
os.startfile("bla2.png")

In [19]:
functions.activate_game_window()
functions.hold_key(functions.game_info.keybindings[GameInterfaceAction.ITEMS])
cv2.imwrite("temp.png", functions.take_screenshot())
functions.release_key(functions.game_info.keybindings[GameInterfaceAction.ITEMS])

In [4]:
def validate_loot(items):
    names = ("Belt", "Blade Talons", "Totemic Mask", "Gemmed Winged Helm", "Thul Rune", "Suwayyah", "Full Rejuvenation Potion")
    qualities = ("Set", "Magic", "Unique", "Grey", "Rune", "Rare", "Common")

    found_all_items = True
    for name, quality in zip(names, qualities):

        found_item = False
        for item in items:
            if item.name == name and item.quality == quality:
                found_item = True
                break

        if found_item is False:
            print("missing item: ", name)
            found_all_items = False

    return found_all_items

game_info.load_font_data()

success = True
acts = ("ACT1", "ACT2", "ACT3", "ACT4", "ACT5")
for act in acts:
    #print(f"testing {act}:")
    #s = time.time()
    items = functions.get_loot(cv2.imread(f"data/{act}.png"), game_info.window_offsets_all)
    #print(time.time()-s)
    act_succesfull = validate_loot(items)
    if act_succesfull is False:
        success = False
    #print()

print(success)



True


In [31]:
game_info.symbol_indices, game_info.sort_indices, game_info.images_widths, game_info.transformation_array, game_info.transformation_array_back, game_info.images_color_counts_core, game_info.sprite_indices, game_info.sprite_indices_length_counter, game_info.num_sprites, game_info.sprite_indices_core, game_info.sprite_indices_length_counter_core, game_info.color_maps_images = load_font_data()

items = functions.get_loot(cv2.imread("ACT5.png"), game_info.window_offsets_all)
for item in items:
    print(item.name)
    
# names = ("Belt", "Blade Talons", "Totemic Mask", "Gemmed Winged Helm", "Thul Rune", "Suwayyah", "Full Rejuvenation Potion")

Blade Talons
Totemic Mask
1
2
3
4
C
Gemmed Winged Helm
Thul Rune
Suwayyah
Life
1363
1419
Mana
562
562
Full Rejuvenation Potion


In [34]:
from helpers import get_transformation_arrays
transformation_array, transformation_array_back = get_transformation_arrays("ACT1")
transformation_array[12,32,40][0]

43

In [36]:
print(transformation_array[72,72,72][0])
print(transformation_array[56,56,56][0])

22
21


In [38]:
transformation_array_back[122]

array([ 20, 124,  40], dtype=uint32)

In [39]:
print(transformation_array[16,36,4][0])

113
